    # Đoạn mã bạn vừa đưa ra sử dụng thư viện dlt (Data Load Tool) — cụ thể là
    helper RESTClient — để thay thế toàn bộ những bước xử lý thủ công
    (như dùng requests, viết vòng lặp while True, và tự tìm link next) ở những bài trước đó
    # thêm token thì sẽ được thêm nhiều request

    2. RESTClient (Trái tim của đoạn code)
    Đây là một "siêu trình duyệt" được dlt thiết kế riêng cho việc lấy dữ liệu từ API.
    base_url: Bạn chỉ định địa chỉ gốc. Mọi yêu cầu sau đó (như /events) sẽ được tự động nối vào sau địa chỉ này.
    auth=BearerTokenAuth(token=API_KEY): Bạn không cần tự viết headers {"Authorization": "Bearer ..."} nữa. dlt sẽ tự động đóng gói Token vào mọi yêu cầu gửi đi một cách chuẩn xác.

    3. HeaderLinkPaginator(links_next_key="next")
    Đây là phần "thông minh nhất" giúp giải quyết lỗi hoặc sự phức tạp khi phân trang:
    GitHub API báo hiệu trang tiếp theo thông qua tiêu đề (Header) của phản hồi HTTP.[4][6]
    Bằng cách khai báo này, bạn đang ra lệnh cho dlt: "Hãy tự nhìn vào Header, tìm cái link nào có chữ 'next', và tự động bấm sang trang đó cho tôi".
    Bạn không cần viết đoạn if 'next' not in response.links: break như trước nữa.

    4. client.paginate("/events")
    Đây là một Generator.
    Thay vì dùng requests.get, bạn dùng client.paginate.
    Nó sẽ tự động thực hiện: Gọi trang 1 -> Trả về dữ liệu -> Tự tìm link trang 2 -> Gọi trang 2... cho đến khi hết dữ liệu thì thôi.

In [ ]:

from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator
from dotenv import load_dotenv
import os

load_dotenv()
url = "https://api.github.com/user"
API_KEY = os.getenv("USER_KEY")
BASE_URL = "https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp"
client = RESTClient(
    base_url=BASE_URL,
    auth=BearerTokenAuth(token=API_KEY),
    paginator=HeaderLinkPaginator(links_next_key="next")
)

for page in client.paginate("/events"):
    print(page)

In [4]:
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator
from dotenv import load_dotenv
import os
import dlt

load_dotenv()
url = "https://api.github.com/user"
API_KEY = os.getenv("USER_KEY")
BASE_URL = "https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp"

@dlt.resource()
def paginated_getter():
    client = RESTClient(
        base_url=BASE_URL,
        auth=BearerTokenAuth(token=API_KEY),
        paginator=HeaderLinkPaginator(links_next_key="next")
    )

    for page in client.paginate("/events"):
        yield page

# paginated_getter nó trả về từng page là 1 list chứa các object
# minh2 duyệt qua từng object
for record in paginated_getter():
    print(record)

{'id': '5822267018', 'type': 'WatchEvent', 'actor': {'id': 5314541, 'login': 'dubechuke', 'display_login': 'dubechuke', 'gravatar_id': '', 'url': 'https://api.github.com/users/dubechuke', 'avatar_url': 'https://avatars.githubusercontent.com/u/5314541?'}, 'repo': {'id': 419661684, 'name': 'DataTalksClub/data-engineering-zoomcamp', 'url': 'https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp'}, 'payload': {'action': 'started'}, 'public': True, 'created_at': '2026-01-15T03:12:36Z', 'org': {'id': 72699292, 'login': 'DataTalksClub', 'gravatar_id': '', 'url': 'https://api.github.com/orgs/DataTalksClub', 'avatar_url': 'https://avatars.githubusercontent.com/u/72699292?'}}
{'id': '5822063392', 'type': 'WatchEvent', 'actor': {'id': 88694450, 'login': 'b0kja85', 'display_login': 'b0kja85', 'gravatar_id': '', 'url': 'https://api.github.com/users/b0kja85', 'avatar_url': 'https://avatars.githubusercontent.com/u/88694450?'}, 'repo': {'id': 419661684, 'name': 'DataTalksClub/data-engine